In [1]:
import re
import pandas as pd

# Define the regular expression for email matching
email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'

# Define the function to extract emails using a finite state accepter model
def extract_emails(text):
    state = 0
    emails = []
    email = ''
    for i, c in enumerate(text):
        if state == 0:  # Initial state
            if c.isalnum() or c == '_' or c == '%':
                email += c
                state = 1
        elif state == 1:  # Username state
            if c.isalnum() or c == '_' or c == '%' or c == '+':
                email += c
            elif c == '.':
                email += c
                state = 2
            elif c == '@':
                email += c
                state = 3
            else:
                email = ''
                state = 0
        elif state == 2:  # Dot state
            if c.isalnum():
                email += c
                state = 1
            else:
                email = ''
                state = 0
        elif state == 3:  # Domain state
            if c.isalnum() or c == '-':
                email += c
            elif c == '.':
                email += c
                state = 4
            else:
                email = ''
                state = 0
        elif state == 4:  # Top-level domain state
            if c.isalpha():
                email += c
            else:
                email = ''
                state = 0
        if state == 4 and i == len(text) - 1:
            emails.append(email)
    return emails

,emails


In [ ]:
# Load the CSV file into a DataFrame
df = pd.read_csv('../emails/emails.csv')

# Apply the regular expression to the content column
df['regex_emails'] = df['contents'].apply(lambda x: re.findall(email_pattern, x))

# Apply the finite state accepter model to the content column
df['fsa_emails'] = df['contents'].apply(extract_emails)

# Combine the results from both approaches
df['emails'] = df.apply(lambda x: list(set(x['regex_emails'] + x['fsa_emails'])), axis=1)

# Flatten the list of emails and remove duplicates
emails_list = list(set([email for sublist in df['emails'] for email in sublist]))

# Create a new DataFrame with the emails
emails_df = pd.DataFrame({'emails': emails_list})
emails_df

In [16]:
from flask import Flask, render_template, request
import json
import os
import re

app = Flask(__name__)

UPLOAD_FOLDER = 'uploads'
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

# Define the regular expression patterns for name, email, and skills matching
name_pattern = r'(\b[A-Z][a-z]*\s[A-Z][a-z]*\b)'
email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
skills_pattern = r'((?i)(Python|Java|JavaScript|CSS|HTML|SQL))'

# Define the function to extract the relevant information from the resume
def extract_resume_info(file_path):
    with open(file_path, 'rb') as f:
        resume = f.read().decode('utf-8', errors='ignore')
    name = re.findall(name_pattern, resume)
    email = re.findall(email_pattern, resume)
    skills = re.findall(skills_pattern, resume)
    return {'name': name, 'email': email, 'skills': skills}

# Define the route for the home page
@app.route('/')
def home():
    return render_template('index_test.html')

# Define the route for uploading the resume file
@app.route('/upload', methods=['POST'])
def upload():
    file = request.files['file']
    if file.filename == '':
        return 'No selected file'
    if file:
        filename = file.filename
        file_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        file.save(file_path)
        data = extract_resume_info(file_path)
        with open('results.json', 'w') as f:
            json.dump(data, f)
        return 'Resume uploaded successfully!'

# Define the route for the parsed resume information
@app.route('/parsed_resume')
def parsed_resume():
    with open('../pdf/results.json') as f:
        data = json.load(f)
    return render_template('parsed_resume.html', data=data)

if __name__ == '__main__':
    app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
[2023-05-13 19:37:10,367] ERROR in app: Exception on / [GET]
Traceback (most recent call last):
  File "c:\Users\johna\AppData\Local\Programs\Python\Python39\lib\site-packages\flask\app.py", line 2190, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\Users\johna\AppData\Local\Programs\Python\Python39\lib\site-packages\flask\app.py", line 1486, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "c:\Users\johna\AppData\Local\Programs\Python\Python39\lib\site-packages\flask\app.py", line 1484, in full_dispatch_request
    rv = self.dispatch_request()
  File "c:\Users\johna\AppData\Local\Programs\Python\Python39\lib\site-packages\flask\app.py", line 1469, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "C:\Users\johna\AppData\Local\Temp\ipykernel_2824\2318685725.py", line 28, in home
    return render_template('index_test.html')
  File "c:\U